In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


In [ ]:
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Display dataset information
print(f"Training Data Shape: {X_train.shape}")
print(f"Test Data Shape: {X_test.shape}")

# Plot sample images
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train[i])
    plt.title(f"Class: {y_train[i][0]}")
    plt.axis("off")
plt.suptitle("Sample CIFAR-10 Images", fontsize=16)
plt.show()


In [ ]:
# Initialize the ImageDataGenerator with basic augmentations
datagen = ImageDataGenerator(
    rotation_range=30,       # Rotate images by up to 30 degrees
    width_shift_range=0.2,   # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,         # Apply shearing transformations
    zoom_range=0.2,          # Zoom in or out
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest'      # Fill pixels after transformations
)

# Select a sample image for augmentation
sample_image = X_train[0]
sample_image = np.expand_dims(sample_image, axis=0)

# Generate augmented images
augmented_images = [datagen.flow(sample_image, batch_size=1).next()[0].astype("uint8") for _ in range(10)]

# Plot augmented images
plt.figure(figsize=(10, 5))
for i, img in enumerate(augmented_images):
    plt.subplot(2, 5, i + 1)
    plt.imshow(img)
    plt.axis("off")
plt.suptitle("Augmented Images", fontsize=16)
plt.show()


In [ ]:
# Install Albumentations
!pip install albumentations


In [ ]:
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations.augmentations.transforms import RandomBrightnessContrast, Blur
from albumentations.core.transforms_interface import ImageOnlyTransform

# Define an augmentation pipeline
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Blur(blur_limit=3, p=0.2),
    A.Rotate(limit=40, p=0.5)
])

# Apply augmentations to a sample image
augmented_image = augmentation_pipeline(image=X_train[0])['image']

# Display original and augmented images
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(X_train[0])
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(augmented_image)
plt.title("Augmented Image")
plt.axis("off")

plt.suptitle("Albumentations Example", fontsize=16)
plt.show()


In [ ]:
# Initialize a new ImageDataGenerator with augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Flow augmented images from the CIFAR-10 dataset
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)

# Define a simple CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using augmented data
model.fit(train_generator, epochs=5, validation_data=(X_test, y_test))
